# HCAN 361 Final Project: Spring 2021

## Access to Acute Ischemic Stroke Care in Southeastern US Hospitals: Implications of Telestroke

## Rationale of analysis:

1. Stroke is a leading cause of death in the US, which is treatable if addressed immediately after it occurs (https://www.cdc.gov/stroke/index.htm)
2. Intravenous recombinant tissue-type plasminogen activator (r-tPA) is a FDA approved treatment for bouts of acute ischemic stroke (AIS), which is often performed under the guidance of a neurologist (https://www.mayoclinicproceedings.org/article/S0025-6196(14)00244-4/fulltext)
3. Some hospitals do not have well-staffed nuerologists available to provide r-tPA when needed, which was why teleStroke was created
4. This analysis will evaluate the impact on Telestroke at the ecological level, and also examine physical access to hospitals in the Southeast with the ability to provide comprehensive stroke care (albeit in-person or through teleStroke).

In [1]:
Use [HCAN361_FinalProj]

--Build provider table for analyses
--drop table #HOSP_STATUS
SELECT A.ID
      ,cast(A.LAT as float) as LAT
	  ,cast(A.LONG as float) as LONG
	  ,A.MNAME
	  ,case 
	   when A.ID in (select ID from dbo.as19svc3 where [STRCHOS]='1')	then '01: Reported Telestroke'
	   when A.ID in (select ID from dbo.as19svc2 where [NEROHOS]='1')	and
			A.ID not in (select ID from dbo.as19svc3 where [STRCHOS]='1')
	        then '02: Reported Neurology, No Telestroke'
	   when A.ID not in (select ID from dbo.as19svc2 where [NEROHOS]='1')	and
	        A.ID not in (select ID from dbo.as19svc3 where [STRCHOS]='1') and
			(
			A.ID in (select ID from dbo.as19svc3 where COFVHOS = '1') or
			A.ID in (select ID from dbo.as19svc3 where EICUHOS = '1') or
			A.ID in (select ID from dbo.as19svc3 where ADTRTHOS = '1') or
			A.ID in (select ID from dbo.as19svc3 where PDISHOS = '1') or
			A.ID in (select ID from dbo.as19svc3 where CHCARHOS = '1') or
			A.ID in (select ID from dbo.as19svc3 where ORPMHOS = '1') or
			A.ID in (select ID from dbo.as19svc3 where OTHTHOS = '1') 
			)
			then '03: Reported Other Telehealth, No Neurology, No Telestroke'
	   else '00: No Telehealth, No Neurology' end as [Status]
	  ,C.County_Name,C.State_Name,C.HRSA_RURAL_DESC
	  ,case when C.[Median_ADI_NatRank]<=25 then '02: Upper Class'
	   when (C.[Median_ADI_NatRank]>25 and C.[Median_ADI_NatRank]<=75) then '01: Middle Class'
	   when C.[Median_ADI_NatRank]>75 then '00: Lower Class'
	   else '99: Not Ranked' end as [ADI]
INTO #HOSP_STATUS
FROM 
dbo.AS19DEM as A
inner join
[dbo].[Provider_Census_Tract] as B
	on A.ID=B.Provider_ID
inner join 
[dbo].[US_Census_Tract_Charac] as C
	on B.GEOID_Data=C.GEOID_Data;


--[STRCHOS]='1' --> hospital reported telestroke --> as19svc3
--[NEROHOS]='1' --> hospital reported neuro --> as19svc2
--> has capacity to service telestroke, but does not
--COFVHOS = '1' --EICUHOS = '1' --ADTRTHOS = '1'
--PDISHOS = '1' --CHCARHOS = '1' --ORPMHOS = '1'
--OTHTHOS = '1'	--as19svc3

--[Status]
--'00: No Telehealth, No Neurology'
--'02: Reported Neurology, No Telestroke'
--'03: Reported Other Telehealth, No Neurology, No Telestroke'
--'01: Reported Telestroke'

select top 10 *
from #HOSP_STATUS
order by State_Name


(6152 rows affected)

(10 rows affected)

Total execution time: 00:00:00.398

ID,LAT,LONG,MNAME,Status,County_Name,State_Name,HRSA_RURAL_DESC,ADI
6530005,34.7192,-86.58,Encompass Rehabilitation Hospital of North Alabama,"00: No Telehealth, No Neurology",Madison,Alabama,00: Urban,01: Middle Class
6530006,30.6964,-88.0798,Mobile Infirmary Medical Center,"00: No Telehealth, No Neurology",Mobile,Alabama,00: Urban,01: Middle Class
6530012,31.2179,-85.3657,Encompass Health Rehabilitation Hospital of Dothan,"00: No Telehealth, No Neurology",Houston,Alabama,00: Urban,00: Lower Class
6530014,33.4635,-86.7767,Select Specialty Hospital-Birmingham,"00: No Telehealth, No Neurology",Jefferson,Alabama,00: Urban,02: Upper Class
6530015,31.2037,-85.3846,Laurel Oaks Behavioral Health Center,"00: No Telehealth, No Neurology",Houston,Alabama,00: Urban,00: Lower Class
6530017,33.2219,-87.543,Taylor Hardin Secure Medical Facility,"00: No Telehealth, No Neurology",Tuscaloosa,Alabama,00: Urban,99: Not Ranked
6530020,33.2081,-87.5377,Mary S Harper Geriatric Psychiatry Center,"00: No Telehealth, No Neurology",Tuscaloosa,Alabama,00: Urban,99: Not Ranked
6530022,31.2159,-85.363,Noland Hospital Dothan,"00: No Telehealth, No Neurology",Houston,Alabama,00: Urban,00: Lower Class
6530023,33.2091,-87.5482,Noland Hospital Tuscaloosa,"00: No Telehealth, No Neurology",Tuscaloosa,Alabama,00: Urban,99: Not Ranked
6530050,31.2963,-86.4948,Andalusia Health,01: Reported Telestroke,Covington,Alabama,01: Rural,01: Middle Class


In [2]:
--Build the population-side table for analyses
select [GEOID_Data], [State_Name],[County_Name], [HRSA_RURAL_DESC],
case when [Median_ADI_NatRank]<=25 then '02: Upper Class'
when ([Median_ADI_NatRank]>25 and [Median_ADI_NatRank]<=75) then '01: Middle Class'
when [Median_ADI_NatRank]>75 then '00: Lower Class'
else '99: Not Ranked' end as [ADI]
into #Pop_Character
from [dbo].[US_Census_Tract_Charac]
where State_Name in ('Alabama','Georgia','Mississippi','Tennessee','Florida');

select top 10 *
from #Pop_Character


(9556 rows affected)

(10 rows affected)

Total execution time: 00:00:00.083

GEOID_Data,State_Name,County_Name,HRSA_RURAL_DESC,ADI
14000US12105012410,Florida,Polk,00: Urban,01: Middle Class
14000US12105012411,Florida,Polk,00: Urban,01: Middle Class
14000US12105012502,Florida,Polk,00: Urban,00: Lower Class
14000US12105012503,Florida,Polk,00: Urban,01: Middle Class
14000US12105012504,Florida,Polk,00: Urban,01: Middle Class
14000US12105012506,Florida,Polk,00: Urban,01: Middle Class
14000US12105012507,Florida,Polk,00: Urban,01: Middle Class
14000US12105012601,Florida,Polk,00: Urban,01: Middle Class
14000US12105012602,Florida,Polk,00: Urban,00: Lower Class
14000US12105012700,Florida,Polk,00: Urban,01: Middle Class


In [3]:
--Simple Physical Access with Telestroke and Neuro

Select Distinct A.*,ISNULL(Closest_Provider,361) AS Closest_Provider_With_Telestroke
into #MapWTeleStroke
from
--Pop Characteristics in region
#Pop_Character A
left join
--Find closest telestroke/neuro facility in region
(
select GEOID_Data, MIN(TOTAL_TRAVELTIME) AS Closest_Provider
from 
[dbo].[Travel_Matrix]
where GEOID_Data in
(select GEOID_Data from #Pop_Character)
and Provider_ID in
(select ID from #HOSP_STATUS where [Status] in ('01: Reported Telestroke','02: Reported Neurology, No Telestroke'))
Group by GEOID_Data
) B
	on A.GEOID_Data=B.GEOID_Data
order by GEOID_Data;

select top 10 *
from #MapWTeleStroke
order by GEOID_Data;

(9556 rows affected)

(10 rows affected)

Total execution time: 00:00:05.403

GEOID_Data,State_Name,County_Name,HRSA_RURAL_DESC,ADI,Closest_Provider_With_Telestroke
14000US01001020100,Alabama,Autauga,00: Urban,01: Middle Class,6.843571203154271
14000US01001020200,Alabama,Autauga,00: Urban,01: Middle Class,5.950789972196525
14000US01001020300,Alabama,Autauga,00: Urban,01: Middle Class,2.915442250855095
14000US01001020400,Alabama,Autauga,00: Urban,01: Middle Class,3.559634007734176
14000US01001020500,Alabama,Autauga,00: Urban,01: Middle Class,5.036663439784612
14000US01001020600,Alabama,Autauga,00: Urban,01: Middle Class,7.242245256976144
14000US01001020700,Alabama,Autauga,00: Urban,01: Middle Class,8.620686006245055
14000US01001020801,Alabama,Autauga,00: Urban,02: Upper Class,12.762006628959126
14000US01001020802,Alabama,Autauga,00: Urban,01: Middle Class,11.706857204085829
14000US01001020900,Alabama,Autauga,00: Urban,01: Middle Class,21.902691464307935


In [5]:
drop table #Map_WO_TeleStroke
Select Distinct A.*,ISNULL(Closest_Provider,361) AS Closest_Provider_WithOut_Telestroke
into #Map_WO_TeleStroke
from
--Pop Characteristics in region
#Pop_Character A
left join
--Find closest neuro facility in region
(
select GEOID_Data, MIN(TOTAL_TRAVELTIME) AS Closest_Provider
from 
[dbo].[Travel_Matrix]
where GEOID_Data in
(select GEOID_Data from #Pop_Character)
and Provider_ID in
(select ID from #HOSP_STATUS where [Status] in ('02: Reported Neurology, No Telestroke'))
Group by GEOID_Data
) B
	on A.GEOID_Data=B.GEOID_Data
order by GEOID_Data;

select top 10 *
from #Map_WO_TeleStroke
order by GEOID_Data;
--Note: Makes underlying assumption that after vetting a hanful of hospitals that hospitals with Telestroke are reporting that they have Neurology Services, but do not have a neurologist on staff at facility

(9556 rows affected)

(10 rows affected)

Total execution time: 00:00:06.216

GEOID_Data,State_Name,County_Name,HRSA_RURAL_DESC,ADI,Closest_Provider_WithOut_Telestroke
14000US01001020100,Alabama,Autauga,00: Urban,01: Middle Class,29.09444595675894
14000US01001020200,Alabama,Autauga,00: Urban,01: Middle Class,28.570595265809867
14000US01001020300,Alabama,Autauga,00: Urban,01: Middle Class,25.86012767735112
14000US01001020400,Alabama,Autauga,00: Urban,01: Middle Class,24.453051622092076
14000US01001020500,Alabama,Autauga,00: Urban,01: Middle Class,22.444211912783267
14000US01001020600,Alabama,Autauga,00: Urban,01: Middle Class,23.73446334413071
14000US01001020700,Alabama,Autauga,00: Urban,01: Middle Class,21.051896330181275
14000US01001020801,Alabama,Autauga,00: Urban,02: Upper Class,29.213601746700427
14000US01001020802,Alabama,Autauga,00: Urban,01: Middle Class,29.717415648753658
14000US01001020900,Alabama,Autauga,00: Urban,01: Middle Class,38.01846170555225


In [6]:
select distinct 
 A.*
,B.Closest_Provider_WithOut_Telestroke
,((B.Closest_Provider_WithOut_Telestroke-A.Closest_Provider_With_Telestroke))/A.Closest_Provider_With_Telestroke as [Percent Increase If No Telestroke]
into #final_Map_Pop
from 
#MapWTeleStroke as A
left join
#Map_WO_TeleStroke as B
	on A.GEOID_Data=B.GEOID_Data
order by GEOID_Data;

select top 10 *
from #final_Map_Pop

(9556 rows affected)

(10 rows affected)

Total execution time: 00:00:00.160

GEOID_Data,State_Name,County_Name,HRSA_RURAL_DESC,ADI,Closest_Provider_With_Telestroke,Closest_Provider_WithOut_Telestroke,Percent Increase If No Telestroke
14000US28075000200,Mississippi,Lauderdale,01: Rural,00: Lower Class,3.475411118517171,3.475411118517171,0
14000US28075000300,Mississippi,Lauderdale,01: Rural,00: Lower Class,4.466126470062882,4.466126470062882,0
14000US28075000400,Mississippi,Lauderdale,01: Rural,00: Lower Class,5.877454434650961,5.877454434650961,0
14000US28075000600,Mississippi,Lauderdale,01: Rural,00: Lower Class,4.613735494399484,4.613735494399484,0
14000US28075000700,Mississippi,Lauderdale,01: Rural,00: Lower Class,7.912373894125135,7.912373894125135,0
14000US28075000800,Mississippi,Lauderdale,01: Rural,00: Lower Class,6.519901171274567,6.519901171274567,0
14000US28075000900,Mississippi,Lauderdale,01: Rural,00: Lower Class,9.015435128262586,9.015435128262586,0
14000US28075001000,Mississippi,Lauderdale,01: Rural,01: Middle Class,11.310804846203196,11.310804846203196,0
14000US28075001101,Mississippi,Lauderdale,01: Rural,01: Middle Class,6.908675867669704,6.908675867669704,0
14000US28075001102,Mississippi,Lauderdale,01: Rural,01: Middle Class,12.079732402434736,12.079732402434736,0


In [7]:
--Create permanent tables for Tableau and ArcGIS
select distinct *
into [HCAN361_25].dbo.Prov_Loc_Group7
from #HOSP_STATUS;

Alter table [HCAN361_25].dbo.Prov_Loc_Group7
Alter Column ID varchar (30) not null;

Alter table [HCAN361_25].dbo.Prov_Loc_Group7
Add constraint pk_Grp7_Prov Primary Key(ID);

select distinct *
into HCAN361_25.dbo.PopMap_Group7
from #final_Map_Pop;

Alter table HCAN361_25.dbo.PopMap_Group7
Alter column GEOID_Data varchar(30) not null;

Alter table HCAN361_25.dbo.PopMap_Group7
Add constraint pk_Grp7_PopMap Primary Key(GEOID_Data);

--#HOSP_STATUS--> Hospital Locations
--#final_Map_Pop--> Pop Map

(6152 rows affected)

(9556 rows affected)

Total execution time: 00:00:00.512

In [8]:
--Table 1: Number of Telestroke Hospitals and as a function of percent of all hospitals within each state

select State_Name
      ,count(distinct id) as [Total Hospitals Statewide]
	  ,count(distinct case when [Status]='01: Reported Telestroke' then id end) as [TeleStroke Hospitals Statewide]
	  ,((count(distinct case when [Status]='01: Reported Telestroke' then id end))*1.00)/((count(distinct id))*1.00) as [Percent TeleStroke Statewide]
	  ,count(distinct case when [HRSA_RURAL_DESC]='00: Urban' then id end) as [Total Hospitals Urban]
	  ,count(distinct case when ([Status]='01: Reported Telestroke' and [HRSA_RURAL_DESC]='00: Urban') then id end) as [TeleStroke Hospitals Urban]
	  ,((count(distinct case when ([Status]='01: Reported Telestroke' and [HRSA_RURAL_DESC]='00: Urban') then id end))*1.00)/((count(distinct case when [HRSA_RURAL_DESC]='00: Urban' then id end))*1.00) as [Percent TeleStroke Urban]
	  ,count(distinct case when [HRSA_RURAL_DESC]<>'00: Urban' then id end) as [Total Hospitals Rural]
	  ,count(distinct case when ([Status]='01: Reported Telestroke' and [HRSA_RURAL_DESC]<>'00: Urban') then id end) as [TeleStroke Hospitals Rural]
	  ,((count(distinct case when ([Status]='01: Reported Telestroke' and [HRSA_RURAL_DESC]<>'00: Urban') then id end))*1.00)/((count(distinct case when [HRSA_RURAL_DESC]<>'00: Urban' then id end))*1.00) as [Percent TeleStroke Rural]
from [HCAN361_25].dbo.Prov_Loc_Group7
where State_Name in ('Alabama','Georgia','Mississippi','Tennessee','Florida')
group by State_Name
order by State_Name;

Warning: Null value is eliminated by an aggregate or other SET operation.

(5 rows affected)

Total execution time: 00:00:00.073

State_Name,Total Hospitals Statewide,TeleStroke Hospitals Statewide,Percent TeleStroke Statewide,Total Hospitals Urban,TeleStroke Hospitals Urban,Percent TeleStroke Urban,Total Hospitals Rural,TeleStroke Hospitals Rural,Percent TeleStroke Rural
Alabama,116,14,0.12068965517241379,68,9,0.13235294117647058,48,5,0.10416666666666666
Florida,253,68,0.26877470355731225,231,64,0.27705627705627705,22,4,0.18181818181818181
Georgia,172,27,0.15697674418604651,105,18,0.17142857142857142,67,9,0.13432835820895522
Mississippi,112,23,0.20535714285714285,36,6,0.16666666666666666,76,17,0.22368421052631578
Tennessee,133,33,0.24812030075187969,79,22,0.27848101265822784,54,11,0.20370370370370370


In [9]:
--Table 2: Median Travel Time With and Without Telestroke
Select Distinct
 State_Name
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY Closest_Provider_With_Telestroke) OVER (PARTITION BY State_Name) AS [Median Closest Travel Time With TeleStroke Statewide]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY Closest_Provider_WithOut_Telestroke) OVER (PARTITION BY State_Name) AS [Median Closest Travel Time Without TeleStroke Statewide]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then Closest_Provider_With_Telestroke end) OVER (PARTITION BY State_Name) AS [Median Closest Travel Time With TeleStroke Urban]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then  Closest_Provider_WithOut_Telestroke end) OVER (PARTITION BY State_Name) AS [Median Closest Travel Time Without TeleStroke Urban]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then Closest_Provider_With_Telestroke end) OVER (PARTITION BY State_Name) AS [Median Closest Travel Time With TeleStroke Rural]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then  Closest_Provider_WithOut_Telestroke end) OVER (PARTITION BY State_Name) AS [Median Closest Travel Time Without TeleStroke Rural]
from
HCAN361_25.dbo.PopMap_Group7
order by State_Name;

(5 rows affected)

Total execution time: 00:00:00.393

State_Name,Median Closest Travel Time With TeleStroke Statewide,Median Closest Travel Time Without TeleStroke Statewide,Median Closest Travel Time With TeleStroke Urban,Median Closest Travel Time Without TeleStroke Urban,Median Closest Travel Time With TeleStroke Rural,Median Closest Travel Time Without TeleStroke Rural
Alabama,20.32661269613938,30.382947530426335,14.666570685806073,19.619856187098314,35.18803783363203,53.56790285204495
Florida,12.048842228287782,16.312954229655663,11.629565985133901,15.702851482282234,36.83027069657001,51.131747522235045
Georgia,15.687213289682166,20.617557036696414,13.689732570367738,16.942679494269036,33.730134097957766,47.02333457929168
Mississippi,17.071605106736495,31.275658796091733,13.224976426128444,16.888722330647425,20.702715235180474,42.0978792258205
Tennessee,16.99732670953837,35.73774458496513,14.035655093245932,29.44543804798561,36.80444361213521,58.18957332446782
